# Assignment-1: Transformations and representations

Team Name: Whatevs

Roll number: 2019101017, 2019101049

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np
from open3d import visualization
import copy
import math
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import time

from numpy import linalg as LA

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

### 1.1

In [2]:
mesh = o3d.io.read_triangle_mesh("data/bunny.ply")
mesh.compute_vertex_normals()
visualization.draw_geometries([mesh], mesh_show_back_face=True)

### 1.2

In [4]:
pcd = mesh.sample_points_uniformly(number_of_points=10000)
pcd.paint_uniform_color([1, 0.706, 0])
visualization.draw_geometries([pcd])

<class 'open3d.cpu.pybind.geometry.PointCloud'>


### 1.3

In [6]:
def customVisual(arr, angle=0):
    viewer = visualization.Visualizer()
    viewer.create_window()
    
    for shape in arr:
        viewer.add_geometry(shape)
        
    opt = viewer.get_render_option()
    opt.show_coordinate_frame = True
    
    ctr = viewer.get_view_control()
    print("Field of view before changing: %.2f" % ctr.get_field_of_view())
    ctr.change_field_of_view(step=angle/5)
    print("Field of view after changing: %.2f" % ctr.get_field_of_view())
    
    viewer.run()
    viewer.destroy_window()

customVisual([pcd], -60)

Field of view before changing: 60.00
Field of view after changing: 5.00


### 1.4 - Scaling

In [7]:
scaled = copy.deepcopy(pcd).scale(2, center=pcd.get_center())

#customVisual([pcd, scaled])
o3d.visualization.draw_geometries([pcd, scaled])

### 1.4 - Rotation

In [8]:
rotated = copy.deepcopy(scaled)
R = mesh.get_rotation_matrix_from_xyz((np.pi / 2, np.pi / 2, 0))
rotated.rotate(R)

#customVisual([scaled, rotated])
o3d.visualization.draw_geometries([scaled, rotated])

### 1.4 - Translation

In [12]:
trans = copy.deepcopy(rotated).translate((0.5, 0.5, 0.5))

#customVisual([rotated, trans])
o3d.visualization.draw_geometries([rotated, trans])

### 1.5

In [13]:
o3d.io.write_point_cloud('results/1_5/bunny.pcd', trans)

True

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


### 2.a.1

In [27]:
def eulerToMat(x, y, z):
    Rx = [[1, 0, 0],
          [0, math.cos(x), -math.sin(x)],
          [0, math.sin(x), math.cos(x)]
         ]
    Ry = [[math.cos(y), 0, math.sin(y)],
          [0, 1, 0],
          [-math.sin(y), 0, math.cos(y)]
         ]
    Rz = [[math.cos(z), -math.sin(z), 0],
          [math.sin(z), math.cos(z), 0],
          [0, 0, 1]
         ]
    
    R = np.dot(Rz, np.dot(Ry, Rx))
    return R

x, y, z = np.pi / 4, 0, np.pi / 2
print("Angles: ""[", x, ", ", y, ", ", z, "]")
print("Calculated Matrix: \n", eulerToMat(x, y, z))
print("Library Matrix: \n", R.from_euler('xyz',[x, y, z]).as_matrix())


Angles: [ 0.7853981633974483 ,  0 ,  1.5707963267948966 ]
Calculated Matrix: 
 [[ 6.12323400e-17 -7.07106781e-01  7.07106781e-01]
 [ 1.00000000e+00  4.32978028e-17 -4.32978028e-17]
 [ 0.00000000e+00  7.07106781e-01  7.07106781e-01]]
Library Matrix: 
 [[ 1.66533454e-16 -7.07106781e-01  7.07106781e-01]
 [ 1.00000000e+00  1.66533454e-16 -1.11022302e-16]
 [ 0.00000000e+00  7.07106781e-01  7.07106781e-01]]


### 2.a.2

In [28]:
def isRotationMatrix(R):
    dotProd = np.dot(np.transpose(R), R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - dotProd)
    
    return n < 1e-6

def matToAngles(est, *args):
    R = args[0]
    assert(isRotationMatrix(R))
    
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    
    if not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
        
    return np.array(est - [x, y, z])

rot = np.array([[0.26200263, -0.19674724, 0.944799],
                [0.21984631, 0.96542533, 0.14007684],
                [-0.93969262, 0.17101007, 0.29619813]])
ans = fsolve(matToAngles, [1, 1, 1], args=(rot))
print("Matrix M")
print("Angles: ", ans)
print("Matrix from Angles: \n", eulerToMat(ans[0], ans[1], ans[2]))


rot2 = np.array([[0, -0.173648178, 0.984807753],
                [0, 0.9848077530, 0.173648178],
                [-1, 0, 0]])
ans2 = fsolve(matToAngles, [1, 1, 1], args=(rot2))
print("\nMatrix N")
print("Angles: ", ans2)
print("Matrix from Angles: \n", eulerToMat(ans2[0], ans2[1], ans2[2]))

Matrix M
Angles:  [0.52359878 1.22173048 0.6981317 ]
Matrix from Angles: 
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

Matrix N
Angles:  [-1.74532926e-01  1.57079633e+00  8.07793567e-28]
Matrix from Angles: 
 [[ 6.12323400e-17 -1.73648178e-01  9.84807753e-01]
 [ 4.94630903e-44  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00 -1.06328843e-17  6.03020831e-17]]


### 2.a.3

It is the phenomenon where rotation about one axis is equivalent to rotating about the other axis before the alignment of the two axes in a parallel direction which itself occurs due to a rotation about the third axis, in the Euler convention. This results in the loss of 1 degree of freedom as the net rotation can be represented by rotations about 2 axes instead of 3.
For example, in the Euler convention, rotation about the Z axis is the same as a previous rotation about the X axis before the rotation about the Y axis that aligns the X and Z axes.

### 2.a.4

Here we visualize a Gimbal Lock by showing that the same 3D rotation can be achieved by using rotations about just 2 axes instead of 3. We rotate the 1st object about the X and Y axes, the 2nd about the Y and Z axes but with the sign reversed for the Z axis rotation and the 3rd about all 3 axes. Then to confirm that all 3 have the same orientation, we translate them so that their centres overlap and see that they are perfect fits with each other.

In [29]:
pcd = o3d.io.read_point_cloud('results/1_5/bunny.pcd')
step = 0.5
pcds = [copy.deepcopy(pcd).translate((-step, 0, 0)), pcd, copy.deepcopy(pcd).translate((step, 0, 0))]
rots = np.multiply([[45, 90, 0], [0, 90, -45], [22.5, 90, -22.5]], np.pi / 180)

viewer = visualization.Visualizer()
viewer.create_window()    
for shape in pcds:
    viewer.add_geometry(shape)        

def rotAnim(viewer, pcd, netRot, divs=100):
    rot = tuple(np.multiply(netRot, 1 / divs))
    
    for axis in range(len(rot)):
        if not rot[axis]:
            continue
        
        rotvec = [0, 0, 0]
        rotvec[axis] = rot[axis]
        R = pcd.get_rotation_matrix_from_xyz(rotvec)
        
        for i in range(divs):    
            pcd.rotate(R)
            viewer.update_geometry(pcd)
            viewer.poll_events()
            viewer.update_renderer()
            
def transAnim(viewer, pcd, divs=5):
    for i in range(abs(divs)):
        pcd.translate((step/divs, 0, 0))
        viewer.update_geometry(pcd)
        viewer.poll_events()
        viewer.update_renderer()
        time.sleep(1)

for idx in range(len(pcds)):
    rotAnim(viewer, pcds[idx], rots[idx])

transAnim(viewer, pcds[0])
transAnim(viewer, pcds[2], -5)
time.sleep(2)

viewer.destroy_window()    

## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

### 2.b.1

The primary reasons why Quaternions are preferred in computer graphics are:
1. Matrix rotations suffer from Gimbal Locks.
2. Quaternions consume less memory and are faster to compute than matrices.
3. Quaternion interpolation using SLERP and SQUAD provide a way to interpolate smoothly between orientations in space.
4. Rotation concatenation using quaternions is faster than combining rotations expressed in matrix form.
5. Converting quaternions to matrices is slightly faster than for Euler angles.
6. Quaternions only require 4 numbers (3 if they are normalized. The Real part can be computed at run-time) to represent a rotation where a matrix requires at least 9 values.

### 2.b.2

In [37]:
# q0 --> qw, q1 --> qx, q2 --> qy, q3 --> qz
def quatToMat(q):
    R = [[q[0] * q[0] + q[1] * q[1] - 0.5, q[1] * q[2] - q[0] * q[3], q[0] * q[2] + q[1] * q[3]],
         [q[0] * q[3] + q[1] * q[2], q[0] * q[0] + q[2] * q[2] - 0.5, q[2] * q[3] - q[0] * q[1]],
         [q[1] * q[3] - q[0] * q[2], q[0] * q[1] + q[2] * q[3], q[0] * q[0] + q[3] * q[3] - 0.5]
        ]
    R = np.multiply(R, 2)   
    return np.array(R)

def rotToQuat(R):
    tr = R.trace()

    if (tr > 0):
        S = math.sqrt(tr + 1.0) * 2;
        qw = 0.25 * S;
        qx = (R[2][1] - R[1][2]) / S;
        qy = (R[0][2] - R[2][0]) / S;
        qz = (R[1][0] - R[0][1]) / S;
    
    elif ((R[0][0] > R[1][1]) and (R[0][0] > R[2][2])):
        S = math.sqrt(1.0 + R[0][0] - R[1][1] - R[2][2]) * 2;
        qx = 0.25 * S;
        qw = (R[2][1] - R[1][2]) / S;
        qz = (R[0][2] + R[2][0]) / S;
        qy = (R[1][0] + R[0][1]) / S;
    
    elif (R[1][1] > R[2][2]):
        S = math.sqrt(1.0 + R[1][1] - R[0][0] - R[2][2]) * 2;
        qy = 0.25 * S;
        qz = (R[2][1] + R[1][2]) / S;
        qw = (R[0][2] - R[2][0]) / S;
        qx = (R[1][0] + R[0][1]) / S;
    
    else:
        S = math.sqrt(1.0 + R[2][2] - R[1][1] - R[0][0]) * 2;
        qz = 0.25 * S;
        qy = (R[2][1] + R[1][2]) / S;
        qx = (R[0][2] + R[2][0]) / S;
        qw = (R[1][0] - R[0][1]) / S;
    
    return [qw, qx, qy, qz]

Rm = np.array([[0, -1, 0],
              [1, 0, 0],
              [0, 0, 1]])

quat = matToQuat(Rm)
retR = quatToMat(quat)
print("Quaternion: ", quat)
print("Matrix: \n", retR)

Quaternion:  [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Matrix: 
 [[ 2.22044605e-16+0.j -1.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 1.00000000e+00+0.j  2.22044605e-16+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  1.00000000e+00+0.j]]


<ipython-input-35-815bd1f49f92>:46: ComplexWarning: Casting complex values to real discards the imaginary part
  vals = np.round(np.array(vals).astype(float))


### 2.b.3

In [42]:
def quatMul(q0, q1):
    x0, y0, z0, w0 = q0
    x1, y1, z1, w1 = q1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 - y1 * z0 + z1 * y0 + w1 * x0,
                     x1 * z0 + y1 * w0 - z1 * x0 + w1 * y0,
                     -x1 * y0 + y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

A = np.array([[0, -1, 0],
              [1, 0, 0],
              [0, 0, 1]])
B = np.array([[6.12323400e-17, -7.07106781e-01,  7.07106781e-01],
              [1.00000000e+00,  4.32978028e-17, -4.32978028e-17],
              [0.00000000e+00,  7.07106781e-01,  7.07106781e-01]])
aq = rotToQuat(A)
bq = rotToQuat(B)

matRes = np.multiply(A, B)
quatRes = quatMul(aq, bq)

print("Quaternion: ", quatRes)
print("Matrix: \n", matRes)

Quaternion:  [-5.55111512e-17  9.23879532e-01 -2.77555756e-17  3.82683432e-01]
Matrix: 
 [[ 0.          0.70710678  0.        ]
 [ 1.          0.         -0.        ]
 [ 0.          0.          0.70710678]]


### 2.b.4

In [11]:
def interpolate(q0, q1, t):
    quats = []
    theta = np.dot(q0, q1)
    
    for i in range(t + 1):
        s = i / t
        slerp = np.multiply(math.sin((1 - s) * theta) / math.sin(theta), q0) + np.multiply(math.sin(s * theta) / math.sin(theta), q1)
        quats.append(slerp)
    
    return quats

def animate(viewer, pcd, quats):
    for quat in quats:
        rot = quatToMat(quat)
        pcd.rotate(rot)
        
        viewer.update_geometry(pcd)
        viewer.poll_events()
        viewer.update_renderer()
        
        pcd.rotate(np.transpose(rot))
        time.sleep(1)
            
A = np.identity(3)
B = np.array([[0, -1, 0],
              [1, 0, 0],
              [0, 0, 1]])

quats = interpolate(rotToQuat(A), rotToQuat(B), 10)

pcd = o3d.io.read_point_cloud('results/1_5/bunny.pcd')
viewer = visualization.Visualizer()
viewer.create_window()    
viewer.add_geometry(pcd)
animate(viewer, pcd, quats)
viewer.destroy_window()

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

### 2.c.2

In [12]:
def findRotationMatrix(omega, theta):
    omega_hat = np.array([[0, -omega[2], omega[1]],
                          [omega[2], 0, -omega[0]],
                          [-omega[1], omega[0], 0]])

    mat1 = np.sin(theta) * omega_hat
    mat2 = (1 - np.cos(theta)) * (omega_hat @ omega_hat)

    return mat1 + mat2 + np.identity(3)

omega = np.array([2, 1, 15])
theta = 4.1364
rot_mat = findRotationMatrix(omega, theta)
print("Rotation Matrix: \n", rot_mat)

Rotation Matrix: 
 [[-348.09417007   15.66913969   45.50128003]
 [  -9.49048181 -352.72816347   24.84727514]
 [  47.17858813   21.49265894   -6.72332235]]


### 2.c.3

In [14]:
def findLogMap(mat):
    trace = mat[0][0] + mat[1][1] + mat[2][2]

    if np.array_equal(mat, np.identity(3)):
        print("Omega is undefined")
        print("Theta: \n", 0)

    elif trace == -1:
        idx = 2 if mat[2][2] != -1 else 1 if mat[1][1] != -1 else 0

        omega = np.array(mat[0][idx], mat[1][idx], mat[2][idx])
        omega[idx][idx] += 1
        c = 1 / ((2 * (1 + mat[idx][idx])) ** 0.5)
        omega *= c

        print("Omega: \n", omega)
        print("Theta: \n", np.pi)

    else:
        theta = np.arccos((trace - 1) / 2)
        c = 1 / (2 * np.sin(theta))
        omega_hat = (mat - mat.T) * c
        omega = [omega_hat[2][1], omega_hat[0][2], -omega_hat[0][1]]

        print("Theta: ", theta)
        print("Omega hat: \n", omega_hat)
        print("Omega: ", omega)
        
mat = np.array([[0.1, -0.9487, 0.3],
                [0.9487, 0., -0.3162],
                [0.3, 0.3162, 0.9]])
findLogMap(mat)

Theta:  1.5707963267948966
Omega hat: 
 [[ 0.     -0.9487  0.    ]
 [ 0.9487  0.     -0.3162]
 [ 0.      0.3162  0.    ]]
Omega:  [0.3162, 0.0, 0.9487]


# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

### 3.a.1

- Octomaps are implemented based on the octree, a hierarchical data structure in which each node represents the space inside a cubic volume. Each node has 8 children.
- Octomaps do not store individual points. Instead, the points are integrated into a maximum compression occupancy map. This makes it more memory efficient.
- The storage is compressed to a few bits per child node in the tree. We can get the node location and voxel size during traversal, and hence, this information need not be stored.
- The splitting is done as follows:
    - When we find partial data, that section is further split into 8 parts.
    - If a part is completely filled, we do not need to further split it into 8 parts.
    - If a part is completely empty, similarly, we do not need to further split it into 8 parts.
- The above method of making child nodes only when required reduces the storage used by an octomap.

### 3.a.2

- A node in the octree is stable if the log-odds value of a voxel reaches lmin (lower bound) or lmax (upper bound).
- If all children of a subtree are stable leaf nodes with the same occupancy state, then the child nodes can be pruned.
- If new measurements contradict the state of a node in the octree, then child nodes are regenerated and updated.
- We get high map compression using the clamping update policy.
- When we find a partially filled node, we explore and update that node, getting finer details about the environment.
- Since modelling and updating are done probabilistically, dynamic objects and environments can be handled directly.

### 3.a.3

- In point cloud representation, free space and unknown areas are not modelled. Further, point clouds cannot directly deal with sensor noise and dynamic objects and environments. Hence, point clouds are suitable in static environments with highly accurate sensors, where unknown regions are not of importance.
- Point clouds also use more memory than octomaps. Memory consumption will increase with the number of measurements and the size of the area mapped. The absence of an upper bound on memory may cause problems.
- Due to the above reasons, it is better to use Octomaps instead. In robot navigation in a large environment that may change dynamically, octomap representation is favourable. For example, navigation and path planning of flying drones.

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


### 3.b.1

- A signed distance function (SDF) is a continuous function that, given a point in 3D space, outputs its distance from the closest surface.
- The sign indicates whether the point is inside or outside the surface.
    - Positive: outside the surface
    - Negative: inside the surface
    - Zero: on the surface
- The surface boundary is hence represented by points whose SDF values are 0.

### 3.b.2

- When we have views from two cameras, we get the images from the point of view of both cameras.
- In order to aggregate views from two cameras, we first need to find the points which are common in the views of both cameras. To find these common points, we use epipolar geometry.
- Once we have these common points, we can do 3D reconstruction or stitch images be correlating these common points.
- The common points are found using the concept of disparity (in stereo vision) or triangulation.

### 3.b.3

- SDF preserves better detail than voxels.
- In the case of voxels, at each grid position, we have only the occupancy value. We need to place the surface at the centre of each edge. Hence, the resulting shape is wavy.

### 3.b.4

- A point cloud is simply a collection of points. It does not store any information regarding surfaces.
- SDFs on the other hand, describe the actual surface

# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/